In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import h5py
import deepdish as dd
import pickle

In [ ]:
w_lya = 1215.67

In [ ]:
# LOAD VUDS

filename = '/home/alberto/almacen/VUDS_and_VVDS/vuds/cesam_vuds_dr1.h5'

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    spec_f = np.array(f['spectra'])
    spec_w = np.array(f['wavelen'])
    z_Arr = np.array(f['zspec'])
    L_Arr = np.array(f['lyalum'])
    EW_Arr = np.array(f['lya_ew'])
    g_Arr = np.array(f['gJAVA_flux'])[:, 0]

mask = (L_Arr > 0) & (z_Arr > 1.9) & (z_Arr < 5)

spec_f = spec_f[mask]
z_Arr = z_Arr[mask]
spec_w = np.ones(spec_f.shape) * spec_w

# LOAD VVDS

filename = '/home/alberto/almacen/VUDS_and_VVDS/vvds/lyalpha_spec.h5'

vvds = dd.io.load(filename)

# Get the data and concatenate with VUDS
spec_f = np.vstack([spec_f, np.array(vvds['fluxes'][8:, :, 0])])
spec_w = np.vstack([spec_w, np.array(vvds['w'])[8:]])
z_Arr = np.concatenate([z_Arr, np.array(vvds['zspec'])[8:]])

In [ ]:
len(np.where(mask)[0]) + 30

In [ ]:
# Offset between each source and desired redshift
spec_w_corrected = spec_w / (1 + z_Arr).reshape(-1, 1)

w_resolution = 0.1
w_start = spec_w_corrected.min()
w_stop = spec_w_corrected.max()
print(f'w interval : ({w_start:0.1f}, {w_stop:0.1f})')
stack_w = np.arange(w_start, w_stop, w_resolution)

# Normalize the rest-frame spectrum
norm_band = [1250, 2750]
norm_band_idx = (stack_w > norm_band[0]) & (stack_w < norm_band[1])

to_stack_f = np.zeros((len(spec_f), len(stack_w)))
for src in range(len(spec_f)):
    to_stack_f[src] = np.interp(
        stack_w, spec_w_corrected[src], spec_f[src], right=0., left=0.
    )
    norm = np.sum(to_stack_f[src][norm_band_idx] *
                  w_resolution) / (norm_band[1] - norm_band[0])
    to_stack_f[src] /= norm

# Sum all the fluxes and divide by how many times the flux is not zero in that w bin
w_weights = np.ones(to_stack_f.shape)
w_weights[to_stack_f == 0.] = 0
mask_w = (w_weights > 0)

stack_f = np.zeros(to_stack_f.shape[1])
stack_minus = np.zeros(to_stack_f.shape[1])
stack_plus = np.zeros(to_stack_f.shape[1])
for w_bin in range(to_stack_f.shape[1]):
    mask = (mask_w[:, w_bin] > 0)
    stack_f[w_bin], stack_minus[w_bin], stack_plus[w_bin] =\
        np.percentile(to_stack_f[:, w_bin][mask], [50, 16, 84])

In [ ]:
# # Save it
# fw_dict = {
#     'wavelength': stack_w,
#     'normalized_flux': stack_f
# }

# pathname = '/home/alberto/cosmos/LAEs/MyMocks/TAU_PROJECT/FIT_STACK_SPECTRUM_LAE/VUDS_stack_files'
# filename = 'Stack_flux_w_dict.pkl'
# with open(f'{pathname}/{filename}', 'wb') as f:
#     pickle.dump(fw_dict, f)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.plot(stack_w, stack_f)

ax.set_ylim((3e-1, 3))
ax.set_xlim(800, 3100)
ax.set_yscale('log')

# plt.savefig('/home/alberto/Desktop/stack', bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))

for src in np.random.choice(range(len(to_stack_f)), 1):
    print(EW_Arr[src])
    print(np.log10(L_Arr[src]))
    # ax.plot(stack_w, to_stack_f[src], alpha=0.4, c='dimgray')

ax.plot(stack_w, stack_f, c='red', label='Stacked spectrum')
ax.fill_between(stack_w, stack_plus, stack_minus, color='r',
                alpha=0.3, edgecolor='none')

# ax.axvline(1500)
# ax.axvline(2000)

ax.legend(fontsize=11)

ax.set_ylim((-2, 6))
ax.set_xlim((700, 3100))

ax.set_xlabel('$\lambda_0\ (\AA)$', fontsize=12)
ax.set_ylabel('$f_\lambda$ (A. U.)', fontsize=12)

plt.savefig('/home/alberto/Desktop/stack.png', bbox_inches='tight', facecolor='white')
plt.show()